## Información

In [69]:
# seat 839 Aitor --------------- https://www.coches.net/segunda-mano/?MakeId=39&MinPrice=4000&MinYear=2012&MaxYear=2022&MinKms=10000&MaxKms=180000
# skoda 214 Aitor --------------- https://www.coches.net/segunda-mano/?MakeId=40&MinPrice=4000&MinYear=2012&MaxYear=2022&MinKms=10000&MaxKms=180000

## Librerías

In [70]:
# %pip install undetected-chromedriver

In [71]:
# %pip install pandas

In [72]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
import random
import time
import undetected_chromedriver as uc
from selenium import webdriver
import re
import json
from datetime import datetime

## Código

In [73]:
def get_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36')
  options.add_argument("--incognito")
  driver = uc.Chrome(options = options)
  driver.set_page_load_timeout(60)
  driver.set_window_size(1366, 768)
  driver.set_window_position(0, 0)
  driver.delete_all_cookies()
  driver.execute_script("localStorage.clear();")
  driver.execute_script("sessionStorage.clear();")
  driver.execute_script("history.go(0);")
  driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
    'source': 'Object.defineProperty(navigator, "webdriver", {get: () => undefined})'
  })
  return driver

In [74]:
def try_get_all_car_info():
    while True:
        try:
            processed_json = get_all_car_info()
            return processed_json
        except:
            print("La función get_all_car_info() falló. Esperando 5 segundos antes de volver a intentarlo...")
            time.sleep(5)

In [75]:
def get_all_car_info():
  # Busca todas las etiquetas 'script' dentro del 'body'
  script_tags = driver.find_elements(By.XPATH, "//body//script")
  script_content = ""

  for tag in script_tags:
      script = tag.get_attribute("innerHTML")
      if '__INITIAL_PROPS__' in script:
          script_content = script
          break
  # extraer la cadena JSON del texto plano
  match = re.search(r'JSON.parse\("(.*?)"\);', script_content)

  json_string = match.group(1)
  json_string = re.sub(r'\\([\\"])', r'\1', json_string) # eliminar las barras invertidas escapadas
  json_string = json_string.replace('\r', '') # eliminar los caracteres "\r"

  # analizar la cadena JSON
  json_data = json.loads(json_string)

  # Procesar json
  keys_to_extract = [
      "color", "fuelTypeId", "fuelType", "id", "km", "make", "makeId", "model", "modelId", "price", "province",
      "provinceId", "title", "transmissionType", "transmissionTypeId", "year"
  ]
  
  processed_json = {key: json_data["ad"].get(key) for key in keys_to_extract}
  processed_json.update({key: json_data["leadData"].get(key) for key in ["brand", "brand_id", "fuel_types", "model_id", "seller_type"]})
  processed_json.update({key: json_data["vehicleInfo"]["specifications"].get(key) for key in ["bodyType", "cubicCapacity", "doors", "hp"]})
  processed_json["url"] = 'https://www.coches.net' + json_data["ad"]["url"]

  return processed_json

In [76]:
def get_links(driver, url):
    actions = ActionChains(driver)
    driver.get(url)
    time.sleep(random.uniform(1, 2))
    try:
        button = driver.find_element(by=By.XPATH, value='/html/body/div/div/div[2]/div/div[2]/div/div/div/footer/div/button[2]')
        actions.move_to_element(button).perform()
        button.click()
        time.sleep(random.uniform(1.2, 1.8))
    except:
        pass
    increment = 100
    position = 0
    while True:
        driver.execute_script("window.scrollTo(0, " + str(increment) + ");")
        elements = driver.find_elements(By.CSS_SELECTOR, ".mt-CardBasic-titleLink")
        time.sleep(random.uniform(0.6, 1.4))
        links = [element.get_attribute('href') for element in elements]
        if position == 0:
            try:
                actions.move_to_element(elements[0]).perform()
            except:
                pass
            position += 1
        else:
            try:
                actions.move_by_offset(random.uniform(-80, 80), random.uniform(-200, 200))
            except:
                pass
        increment += 300
        if len(elements) >= 30 or increment >= 10000:
            break
    links = [element.get_attribute('href') for element in elements]
    return links

In [77]:
def get_car(driver, url):
    car = {}
    actions = ActionChains(driver)
    driver.get(url)
    time.sleep(random.uniform(1, 1.5))
    try:
        button = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div/div[2]/div/div[3]/div/div/div/footer/div/button[2]')
        actions.move_to_element(button).perform()
        button.click()
        time.sleep(random.uniform(1.2, 1.8))
    except:
        pass
    car['url'] = url
    car['datetime'] = datetime.now()
    car['datetime'] = car["datetime"].strftime('%Y-%m-%d %H:%M:%S')
    #-----------------------------------------------------
    car['resto_de_datos'] = try_get_all_car_info()
    #-----------------------------------------------------
    try:
        actions.move_by_offset(random.uniform(-60, 60), random.uniform(-200, 200))
    except:
        pass
    time.sleep(random.uniform(2, 4))
    return car

In [78]:
def get_cars(driver, urls):
    cars = []
    for url in urls:
        if url != None:
            car = get_car(driver, url)
            cars.append(car)
    return cars


In [79]:
brands_and_pages = [
    {
        "makeId": 39,
        "init_page": 11,
        "pages": 30
    },
    {
        "makeId": 40,
        "init_page": 1,
        "pages": 8
    },
]
nombre_archivo = 'datos.json'

In [80]:
def build_link(make_id, page):
    return f'https://www.coches.net/segunda-mano/?MinPrice=4000&MinYear=2012&MaxYear=2022&MinKms=10000&MaxKms=180000&MakeId={make_id}&pg={page}'

In [81]:
driver = get_driver()
cars = []

with open(nombre_archivo, 'r') as archivo_json:
    cars = json.load(archivo_json)

for bran_page in brands_and_pages:
    for page in range(bran_page["init_page"], bran_page["pages"] + 1):
        print(page)
        links = get_links(driver, build_link(bran_page["makeId"], page))
        for i in range(len(links)):
            if links[i] is not None:
                links[i] = links[i].replace("#", "")

        cars_info = get_cars(driver, links)
        cars.extend(cars_info)

        # Escribir la lista en el archivo JSON
        with open(nombre_archivo, 'w') as archivo_json:
            json.dump(cars, archivo_json)

La función get_all_car_info() falló. Esperando 5 segundos antes de volver a intentarlo...
La función get_all_car_info() falló. Esperando 5 segundos antes de volver a intentarlo...
La función get_all_car_info() falló. Esperando 5 segundos antes de volver a intentarlo...
La función get_all_car_info() falló. Esperando 5 segundos antes de volver a intentarlo...
La función get_all_car_info() falló. Esperando 5 segundos antes de volver a intentarlo...
La función get_all_car_info() falló. Esperando 5 segundos antes de volver a intentarlo...
La función get_all_car_info() falló. Esperando 5 segundos antes de volver a intentarlo...
La función get_all_car_info() falló. Esperando 5 segundos antes de volver a intentarlo...
La función get_all_car_info() falló. Esperando 5 segundos antes de volver a intentarlo...
La función get_all_car_info() falló. Esperando 5 segundos antes de volver a intentarlo...
La función get_all_car_info() falló. Esperando 5 segundos antes de volver a intentarlo...
La función